In [2]:
!pip install torch

In [80]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, datasets, models
import transformers

##Learning Layers in PyTorch

In [10]:
# Some gyan about fully connected layer

# This is a fully connected layer or dense layer which has two main arguments in_features and out_features.
# The in_feature argumnet specifies that it expects input of share (n1, n2,  . . . , n_k) where n_k is
# equal to in_features. As the dense layer applies on the last dimension of the input, and after applying
# the shape of the ouput would be (n1, n2,  . . . , out_features)

fc_layer = nn.Linear(in_features = 100, out_features = 10)

# If we want to apply activation function with this layer, we can wrap it into nn.sigmoid or nn.relu etc. Like nn.relu(fc_layer(input))

# If we want to find the parameter of the layer, one way is fc_layer.parameters() its a generator object
# and contains two items weights and bias, this methods paramters() is valid for models too, like if i do
# model.parameter() that would also make a generator object of all parameter in the model.
# But for fully connected layer like above, we can use fn_layer.weight and fn_laer.bias to get the weight
# and bias of the dense layer

# try
for param in fc_layer.parameters():
  print(param)
  print("next param now")

print("weight of fc_layer: ", fc_layer.weight)
print("bias of fc_layer: ", fc_layer.bias)

# if we want to change intializer the weights in a specific way like he_initializer, lecun etc,
# which was an argument with tf.Dense in tensorflow. It can be done as follows:


# this is he_initialization, once this is run the weights are itself updated, nonlinearity arguments
# sggests that this layer will be wrapped with relu activation thus so that the initialization should
# be adjusted with that

fc1 = nn.Linear(5, 2)
nn.init.kaiming_normal_(fc1.weight, mode='fan_in', nonlinearity='relu')

# this is xavier initilization
nn.init.xavier_uniform_(fc1.weight, gain=nn.init.calculate_gain('relu'))

# also, there isn't a constraint on choosing only the well known initializers, you can also
# define the intilizer in a simple way with the initializer of your choice, for example

fc1 = nn.Linear(5, 2)
fc1.weight = nn.Parameter(torch.randn((5, 2)))
fc1.bias = nn.Parameter(torch.zeros((2)))

# Note: the fc1.weight update works without nn.Parameter wrapper but fc1.bias update does not work wihthout it.


In [10]:
# brief on conv2D layer

# This is a 2D convolution layer which is commonly used in convolutional neural networks (CNNs)
# for image processing tasks. It has several key arguments, the most important being in_channels,
# out_channels, kernel_size, stride, and padding. in_channels specifies the number of input channels
# (depth) that the layer expects, like 3 for RGB images. out_channels refers to the number of filters
#  (or kernels) applied to the input. Each filter generates one output channel.
# kernel_size specifies the size of the convolutional kernel (filter), typically as a
# tuple (height, width) or a single integer if both dimensions are equal.
# stride controls the step size of the convolution as it moves over the input.
# padding adds zero-padding around the input to control the spatial size of the output.
conv_layer = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(4, 4), stride=1, padding=1)

# This layer applies the convolution operation on the input tensor, and if your input has a shape of
# (batch_size, 3, height, width), the output after this layer will be (batch_size, 16, height, width),
# as we have set out_channels to 16 and padding=1 to maintain the spatial dimensions.
# If we want to apply an activation function like ReLU or Sigmoid after the convolution, we can chain it like this:
conv_output = nn.ReLU()(conv_layer(torch.randn(10, 3, 20, 20)))

# To inspect the parameters of the convolutional layer (like weights and biases), we can use conv_layer.parameters(),
# which, like the Linear layer, returns a generator object with two elements: the weights (filters) and biases for each filter.
# for param in conv_layer.parameters():
      # print(param)
      # print("next param now")

# Specifically, you can access the convolution layer's filters (weights) and bias directly as follows:
print("Weight of conv_layer (filters): ", conv_layer.weight)
# the dimesnion of conv layer.weight is (out_channels, in_channels, filter_height, filter_eidth)

print("Bias of conv_layer: ", conv_layer.bias)
# it's a single dimensional vector with size equal to number of out_channels, the the above example its torch.Size([16])

# Like fully connected layers, you can customize the initialization of the convolution filters. By default, PyTorch
# uses Kaiming initialization for convolutional layers, but you can change it to another initializer if needed.
# For example, you can apply He initialization (he is implemented in two ways uding the kaiming
# , using normal or using uniform distribution ) (which is good for layers with ReLU activations):
# nn.init.kaiming_normal_(conv_layer.weight, mode='fan_in', nonlinearity='relu')

# Or Xavier initialization:
# nn.init.xavier_uniform_(conv_layer.weight, gain=nn.init.calculate_gain('relu'))
# Note: gain=nn.init.calculate_gain('relu') is just to adjust the weights for relu activation

# You can also define your custom initialization logic:
# conv_layer.weight = nn.Parameter(torch.randn_like(conv_layer.weight)) conv_layer.bias = nn.Parameter(torch.zeros_like(conv_layer.bias))

# Note: Similar to the fully connected layer, while the weights can be updated directly without wrapping in nn.Parameter, the biases require it to work correctly.


Weight of conv_layer (filters):  Parameter containing:
tensor([[[[-5.7463e-03,  1.2241e-01,  4.4320e-02,  1.3226e-01],
          [-3.6728e-02, -3.1053e-02,  8.4526e-02,  1.8769e-02],
          [ 1.3516e-02,  7.2797e-03, -1.1403e-01,  1.9078e-02],
          [-9.8505e-02, -3.8978e-02, -6.7833e-04,  1.8371e-02]],

         [[ 3.6775e-02, -4.3062e-02,  1.2054e-01, -6.6434e-02],
          [-3.3405e-02, -1.1723e-01,  1.1306e-01,  1.0177e-01],
          [ 1.1857e-01,  1.1485e-01,  1.0110e-02, -4.9143e-03],
          [-1.0549e-01,  2.7721e-02,  5.2446e-02, -2.4568e-02]],

         [[-1.8040e-03, -1.3481e-01, -4.5619e-02,  1.5098e-02],
          [ 4.3307e-02,  8.1590e-02,  8.9634e-02,  9.7663e-02],
          [ 2.8717e-02, -6.8920e-02, -1.0548e-02, -9.0130e-02],
          [-1.0639e-02, -2.8493e-02,  5.1148e-02, -4.3359e-02]]],


        [[[-9.4359e-02,  1.4711e-02,  1.3091e-02, -1.1513e-01],
          [-1.0262e-01, -7.1928e-02, -3.7231e-02, -1.0603e-04],
          [ 1.2334e-01, -7.9252e-02,  7.2

In [9]:
# Some gyan about 2D Pooling Layer

# Pooling layers are used to downsample the input, reducing its spatial dimensions while
# retaining important information. This helps make the model more computationally efficient
# and less prone to overfitting. There are two main types of pooling layers in
# PyTorch: MaxPool2d (max pooling) and AvgPool2d (average pooling).
# The key arguments of a pooling layer are kernel_size, stride, and padding:
# - kernel_size specifies the size of the window (typically (height, width)) over which pooling is applied.
# - stride defines how much the pooling window moves across the input. If not specified, it defaults to the value of kernel_size.
# - padding adds zero-padding around the input before pooling (typically not used in pooling layers, but it can be).
# Example of Max Pooling:
# maxpool_layer = nn.MaxPool2d(kernel_size=2, stride=2)

# In this case, the MaxPool2d layer applies max pooling over a 2x2 window, moving with a stride of 2,
# which effectively reduces the spatial dimensions of the input by half. If the input shape is
# (batch_size, channels, height, width), the output shape after pooling will be (batch_size, channels, height//2, width//2).
# Similarly, we can use Average Pooling (AvgPool2d) to compute the average value in each pooling window:
# avgpool_layer = nn.AvgPool2d(kernel_size=2, stride=2)

# To apply pooling to an input tensor:
# pooled_output = maxpool_layer(input)

# As with convolutional layers, pooling layers have no trainable parameters like weights or biases.
# Their role is purely to downsample the input, so there’s no need to call .parameters().
# A common practice is to apply pooling after convolutional layers to reduce the spatial dimensions
# while preserving the depth (number of channels). Pooling helps make the network more computationally
# efficient while maintaining the most important features.
# Here’s an example of applying both convolution and max pooling together:
# conv_layer = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
# maxpool_layer = nn.MaxPool2d(kernel_size=2, stride=2)

# After a convolutional operation:
# conv_output = conv_layer(input) # the dimension of output layer (batch_size, in_channels, new_height, new_width)

# Apply max pooling to reduce spatial dimensions:
# pooled_output = maxpool_layer(conv_output)

# Note that pooling is typically used after an activation function (like ReLU) to preserve non-linearity:
# relu_output = nn.ReLU()(conv_output) pooled_output = maxpool_layer(relu_output)

# Like in the convolution layer, if you want to inspect the effects of pooling on an input, you can print
# the shape of the input and output tensors before and after pooling:
# print("Shape before pooling: ", conv_output.shape) print("Shape after pooling: ", pooled_output.shape)

torch.Size([16])

In [12]:
# Some gyan about transitioning from Conv2D to a Fully Connected Layer

# In Convolutional Neural Networks (CNNs), after several convolutional
# and pooling layers, the output tensor still retains a 3D structure
#  (i.e., with dimensions corresponding to height, width, and depth/channels).
# the order being (channels, height, width)
#  However, to apply a fully connected layer (or nn.Linear), this 3D output
#  needs to be flattened into a 1D vector. This allows the fully connected
#  layer to treat the flattened tensor as a simple vector of inputs, just
#  like in traditional feedforward networks.
# To understand the transition, let's break it down:
# Output of Conv/Pooling Layer: After a series of convolutional and pooling
# layers, the output typically has the shape (batch_size, channels, height, width).
# For example, say we have a tensor of shape (batch_size=32, channels=16, height=8, width=8).

# Flattening the Output: To connect this 3D tensor to a fully connected layer, you
# first need to "flatten" it, meaning you collapse the (channels, height, width)
# dimensions into a single 1D vector. This can be done using torch.flatten() or
# reshaping the tensor manually using .view() or .reshape() methods.

# --code:

conv_output = torch.randn(32, 16, 8, 8)  # Example output of a conv layer
flattened_output = conv_output.view(32, -1)  # Flattening the tensor (32, 16*8*8)
# The flattened tensor will now have shape (batch_size, 16*8*8). In this case, (32, 1024).

# Fully Connected Layer After Flattening: Now that we have flattened the output, we can pass
# it to a fully connected (nn.Linear) layer. The number of input features (in_features) for
# this layer should match the size of the flattened vector (i.e., 16 * 8 * 8 = 1024 in this example).
#  The number of output features (out_features) is the number of neurons you want in the next layer,
#  depending on your task (e.g., 10 for a 10-class classification problem).


fc_layer = nn.Linear(in_features=16*8*8, out_features=10)  # Example fully connected layer

# Science Behind the Transition:

# Why Flatten? The convolutional layers capture spatial features (edges, textures, shapes)
# from the input, and the output tensor retains this spatial structure. However, fully
#  connected layers expect a simple vector of features. Flattening helps "unroll" the
#  learned spatial features into a 1D vector so that the fully connected layer can process
#  them for the final decision-making part of the network (e.g., classification).

# Why Fully Connected Layer? Once the convolutional layers have distilled the spatial
# information into meaningful high-level features, the fully connected layers serve as a
# "classifier" or "decision-maker" that takes these abstracted features and assigns
# them to the correct class, or performs regression if it's a different task.

# Putting It All Together: Here's an example of how you'd go from a convolutional layer to a fully connected layer in a simple CNN:


class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv_layer = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # After conv and pooling, the output size is (batch_size, 16, 8, 8)
        self.fc_layer = nn.Linear(in_features=16*8*8, out_features=10)

    def forward(self, x):
        x = self.conv_layer(x)  # Conv layer
        x = self.pool(x)        # Pooling layer
        x = x.view(x.size(0), -1)  # Flattening the output (batch_size, 1024)
        x = self.fc_layer(x)    # Fully connected layer
        return x

# In this example, the conv_layer processes the input, the pool downscales it,
# and then the output is flattened to feed into the fully connected layer. This
#  transition allows us to combine feature extraction (via convolutions) with
#  classification (via fully connected layers).

torch.Size([10, 3, 10, 10])

## LEARNING ABOUT RETRIEVING INDIVIDUAL LAYERS AND MAKING REQUIRED CHANGES

In [59]:
# Some gyan on retrieving individual layers of a pytorch models

# Let's first start with a convolution model built by subclassing nn.Module

class MyCustomModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 8, kernel_size = 4)
    self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = 4)
    self.conv3 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 4)
    self.pool = nn.MaxPool2d(kernel_size = 4, stride = 4)
    self.fc1 = nn.Linear(in_features = 128, out_features = 56)
    self.fc2 = nn.Linear(in_features = 56, out_features = 10)

  def forward(self, x):
    x = self.conv1(x)
    x = self.pool(x)
    x = torch.relu(x)
    x = self.conv2(x)
    x = self.pool(x)
    x = torch.relu(x)
    x = self.conv3(x)
    x = self.pool(x)
    x = torch.relu(x)
    x = x.view(x.size(0), -1)
    x = self.fc1(x)
    x = torch.relu(x)
    x = self.fc2(x)
    return x

custom_model = MyCustomModel()

# we can checkout all the layers of this model as follows:
for layer in custom_model.named_children():
  print(layer)

# the reposponse is:
# ('conv1', Conv2d(3, 8, kernel_size=(4, 4), stride=(1, 1)))
# ('conv2', Conv2d(8, 16, kernel_size=(4, 4), stride=(1, 1)))
# ('conv3', Conv2d(16, 32, kernel_size=(4, 4), stride=(1, 1)))
# ('pool', MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False))
# ('fc1', Linear(in_features=128, out_features=56, bias=True))
# ('fc2', Linear(in_features=56, out_features=10, bias=True))

# From the response above, I am able to see all the layers, however I can not understand
# the flow from input to output, as this pooling layer is used many times which is unclear
# from here. Note: It actually picks just layers only, like if i try to confuse it by
# initialiting self.my_layer = "just to confuse", it will not call it as a layer as expected.
# Next we will learn picking up a layer from here and manipulating it:

# From printing the custom_model.named_childeren(), we know that the conv layers with names conv1,
# conv2, conv3 are there, lets say I want to manipulate the weights of it:

with torch.no_grad():
  custom_model.conv1.weight *= 2
  custom_model.conv2.bias = nn.Parameter(torch.zeros(custom_model.conv2.bias.shape))
  custom_model.fc1.weight = nn.Parameter(torch.randn(custom_model.fc1.weight.shape))

for param in custom_model.conv2.parameters():
  param.requires_grad = False




('conv1', Conv2d(3, 8, kernel_size=(4, 4), stride=(1, 1)))
('conv2', Conv2d(8, 16, kernel_size=(4, 4), stride=(1, 1)))
('conv3', Conv2d(16, 32, kernel_size=(4, 4), stride=(1, 1)))
('pool', MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False))
('fc1', Linear(in_features=128, out_features=56, bias=True))
('fc2', Linear(in_features=56, out_features=10, bias=True))


In [60]:
# Now we would learn how to checkout the flow of the model, i.e from input to output what path it goes through

# One simple way is to print the shape and layer name within the forward function while definig the model.
# That is too simple to discuss, lets look at a better and flxible way of doing this, using hook. Let's see how
# that can be done

def print_hook(module, input, output):
  print(f"{module}: \nOutput shape: {output.shape}\n")

# Register hooks on layers
for layer in custom_model.named_children():
  layer[1].register_forward_hook(print_hook)

# Now, we can pass the data though the model to trigger the hooks
x = torch.randn(1, 3, 224, 224)
output = custom_model(x)

Conv2d(3, 8, kernel_size=(4, 4), stride=(1, 1)): 
Output shape: torch.Size([1, 8, 221, 221])

MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False): 
Output shape: torch.Size([1, 8, 55, 55])

Conv2d(8, 16, kernel_size=(4, 4), stride=(1, 1)): 
Output shape: torch.Size([1, 16, 52, 52])

MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False): 
Output shape: torch.Size([1, 16, 13, 13])

Conv2d(16, 32, kernel_size=(4, 4), stride=(1, 1)): 
Output shape: torch.Size([1, 32, 10, 10])

MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False): 
Output shape: torch.Size([1, 32, 2, 2])

Linear(in_features=128, out_features=56, bias=True): 
Output shape: torch.Size([1, 56])

Linear(in_features=56, out_features=10, bias=True): 
Output shape: torch.Size([1, 10])



In [79]:
# Doing the above steps for pretranied models

# Load a pre-trained ResNet model
model = models.resnet18(pretrained=True)

# Print the model architecture
# print(model)


# for name, layer in model.named_children():
#   print(name)
#   print(layer)
#   print(" ")
#   print(" ")

# now in order to make changes to any layer, like conv1
# with torch.no_grad():
#   model.conv1.weight *= 2.0
#   model.fc.weight = nn.Parameter(torch.randn(model.fc.weight.shape))

# # Also, getting to know the flow is also straightforward
def print_hook(module, input, output):
  print(f"{module}: \nOutput shape: {output.shape}\n")

# # Register hooks on layers
for layer in model.named_children():
  layer[1].register_forward_hook(print_hook)

# Now, we can pass the data though the model to trigger the hooks
x = torch.randn(10, 3, 224, 224)
output = model(x)


# Also we see here, there are many blocks which are made of nn.Sequential, we discuss about it in the next cell.

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False): 
Output shape: torch.Size([10, 64, 112, 112])

BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True): 
Output shape: torch.Size([10, 64, 112, 112])

ReLU(inplace=True): 
Output shape: torch.Size([10, 64, 112, 112])

MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False): 
Output shape: torch.Size([10, 64, 56, 56])

Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNor

## LEARNING NN.SEQUENTIAL

In [ ]:
# nn.Sequential in PyTorch is a convenient way to define models where the layers are stacked sequentially,
# one after the other. It simplifies the process of creating a model, especially when the layers are executed
# in a straightforward manner, meaning each layer’s output is passed as the input to the next layer.

# Basic Idea
# Instead of subclassing nn.Module and defining the forward method manually, you can stack layers using
# nn.Sequential, which automatically assumes that the input will be passed through the layers in the
# order they are defined.

# Key Features of nn.Sequential:
# Simplified Layer Stacking: You can define a model by simply listing the layers in the order they are applied.
# Automatic Forward Pass: The forward pass is implicitly defined as the input passes through all layers sequentially.
# Layer Manipulation: You can easily access and manipulate individual layers.
# Example: Basic Usage
# Let’s say you want to define a neural network with:

# A fully connected (dense) layer (nn.Linear),
# A ReLU activation function (nn.ReLU),
# Another fully connected layer.
# You can create this simple model using nn.Sequential:


import torch
import torch.nn as nn

# Define a model using nn.Sequential
model = nn.Sequential(
    nn.Linear(4, 10),  # Input features = 4, Output features = 10
    nn.ReLU(),         # Activation function
    nn.Linear(10, 2)   # Input features = 10, Output features = 2
)

# Print the model architecture
print(model)

# Sequential(
#   (0): Linear(in_features=4, out_features=10, bias=True)
#   (1): ReLU()
#   (2): Linear(in_features=10, out_features=2, bias=True)
# )


# Forward Pass
# Now, if you input a tensor to the model, it will automatically go through the layers in order:

# Create a random input tensor of shape (batch_size, input_features)
input_tensor = torch.randn(3, 4)  # Batch size = 3, Input features = 4

# Forward pass through the model
output = model(input_tensor)

print(output)

# tensor([[-0.0314,  0.2432],
#         [-0.1920,  0.2143],
#         [-0.0284,  0.0652]], grad_fn=<AddmmBackward0>)


# Custom Layer Names
# If you want to give specific names to each layer, you can use an OrderedDict to specify both the names and the layers:


# from collections import OrderedDict

# # Define a model using nn.Sequential with named layers
# model = nn.Sequential(OrderedDict([
#     ('fc1', nn.Linear(4, 10)),  # First fully connected layer
#     ('relu', nn.ReLU()),        # ReLU activation
#     ('fc2', nn.Linear(10, 2))   # Second fully connected layer
# ]))

# Print the model architecture
print(model)
# Accessing Layers in nn.Sequential
# You can access individual layers in nn.Sequential using indices, just like accessing elements in a list:

# Access the first layer (fc1)
print(model[0])  # This will print nn.Linear(4, 10)

# Access the activation function (ReLU)
print(model[1])  # This will print nn.ReLU()

# Modify or replace a layer
model[0] = nn.Linear(4, 20)  # Change the first layer to have 20 output features

# Print updated model
print(model)

## LEARNING LEARNING RATE SCHEDULER

In [ ]:
# Now, we learn how to schedule learning rate of the model in pytorch

# A learning rate scheduler makes changes to the learning rate during the training process,
# makes the training attain different leraning rates as per the plan

# Lets dicuss three types of scheduler, will provide the entrire custom loop for two of them

# StepLR: Reduces the learning rate by a factor every few epochs.
# ExponentialLR: Decays the learning rate exponentially.
# ReduceLROnPlateau: Reduces the learning rate when a metric stops improving.


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Define a simple model for demonstration
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc = nn.Linear(10, 2)

    def forward(self, x):
        return self.fc(x)

# Dummy dataset and dataloader
train_data = torch.randn(100, 10)
train_labels = torch.randint(0, 2, (100,))
train_loader = DataLoader(list(zip(train_data, train_labels)), batch_size=10)

# Define loss function and model
model = MyModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Define StepLR scheduler: reduce lr by 0.1 every 10 epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1) # so, after every 10 epochs the new learnng rate is gamma*existing_learning_rate

# Training loop
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        optimizer.zero_grad()  # Zero gradients

        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update parameters

        running_loss += loss.item()

    # Step the scheduler at the end of each epoch
    scheduler.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Learning Rate: {scheduler.get_last_lr()[0]}')

# Everythings in training loop remain same, just that after every epoch we just run scheduler.step()
# which upodates the info in the scheudler and scheduler make necssary changes when requried, i.e for
#  stepLR it will make the learning_rate as 0.1*learnig_rate after 10 epochs.


# Lets now jump into the ReduceLROnPlateau, its discussed in next section

In [81]:
# The ReduceLROnPlateau learning rate scheduler is designed to adjust the learning rate dynamically
# when a specific metric (often validation loss) stops improving. Instead of reducing the learning
# rate after a fixed number of epochs, like in StepLR, this scheduler reduces the learning rate when
# the monitored metric plateaus—i.e., when the metric (like validation loss) does not improve for a
# certain number of epochs. This allows the model to continue learning effectively when improvement
# slows down, without wasting time on a high learning rate that might already be too aggressive.

# How ReduceLROnPlateau Works:
# Monitored Metric: Typically, the validation loss is monitored, though it can be any other metric.
# Patience: The scheduler waits for a defined number of epochs (patience) without improvement in the monitored metric before reducing the learning rate.
# Factor: The learning rate is reduced by a certain factor (e.g., by multiplying it by 0.1 or another value).
# Mode: The scheduler can operate in either min mode (to reduce the learning rate when the metric stops decreasing, which is typical for loss) or max mode (to reduce when the metric stops increasing, for metrics like accuracy).
# Parameters:
# mode='min': The scheduler reduces the learning rate when the validation loss does not decrease.
# factor=0.1: When triggered, the learning rate is multiplied by this factor.
# patience=5: The scheduler waits for 5 epochs of no improvement before reducing the learning rate.
# verbose: Whether or not to print a message when the learning rate is reduced.
# threshold: The amount of improvement considered significant.
# cooldown: The number of epochs to wait after the learning rate has been reduced before monitoring again.

# In the Training Loop:
# In your training loop, ReduceLROnPlateau is used to adjust the learning rate based on the validation loss. Here's a breakdown of how it operates within your code:

# Key Steps:
# Training Phase:

# Each epoch, the model is trained on the training data, and the training loss is accumulated.
# The optimizer updates the model's weights using backpropagation and the learning rate determined by the current state of the optimizer.

# Validation Phase:

# After training, the model is evaluated on the validation dataset to compute the validation loss (val_loss).
# This loss is the key metric being monitored to determine if the learning rate should be reduced.
# Scheduler Step:

# At the end of each epoch, the scheduler is stepped using the validation loss: scheduler.step(val_loss).
# If the validation loss doesn't improve for patience consecutive epochs, the learning rate is reduced by
# multiplying it by factor. For example, if the current learning rate is 0.1 and factor is 0.1, the
# learning rate will be reduced to 0.01. The learning rate is updated dynamically as training progresses.

# Why Use ReduceLROnPlateau:
# In deep learning, models often hit points where the learning stagnates (loss doesn't decrease or decreases
# very slowly). A high learning rate might prevent the model from fine-tuning, while a lower learning rate
# can help it converge to a better minimum. ReduceLROnPlateau helps manage this by reducing the learning
# rate only when it's needed, making the training process more efficient.

# Now the training loop:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Same model as before
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc = nn.Linear(10, 2)

    def forward(self, x):
        return self.fc(x)

# Dummy datasets and dataloaders
train_data = torch.randn(100, 10)
train_labels = torch.randint(0, 2, (100,))
train_loader = DataLoader(list(zip(train_data, train_labels)), batch_size=10)

val_data = torch.randn(40, 10)
val_labels = torch.randint(0, 2, (40,))
val_loader = DataLoader(list(zip(val_data, val_labels)), batch_size=10)

# Define loss function and model
model = MyModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Define ReduceLROnPlateau scheduler: reduce lr by 0.1 if no improvement in 5 epochs
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

# Function to evaluate on validation set
def validate(model, val_loader):
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    return val_loss / len(val_loader)

# Training loop
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        optimizer.zero_grad()  # Zero gradients

        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update parameters

        running_loss += loss.item()

    # Validation phase
    val_loss = validate(model, val_loader)

    # Step the scheduler based on validation loss
    scheduler.step(val_loss)

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Val Loss: {val_loss}, Learning Rate: {optimizer.param_groups[0]["lr"]}')


# We can also create the custom learning rate scheduler, it can be done by subclassing _LRScheduler, we would need to
# look at the structure of this class to implement that, but it could be easily found online.


Epoch 1/30, Loss: 0.7788966029882431, Val Loss: 0.7355731874704361, Learning Rate: 0.1
Epoch 2/30, Loss: 0.6917988240718842, Val Loss: 0.7289204001426697, Learning Rate: 0.1
Epoch 3/30, Loss: 0.6631533682346344, Val Loss: 0.7389393299818039, Learning Rate: 0.1
Epoch 4/30, Loss: 0.6543763160705567, Val Loss: 0.7502733469009399, Learning Rate: 0.1
Epoch 5/30, Loss: 0.6516462564468384, Val Loss: 0.7594905197620392, Learning Rate: 0.1
Epoch 6/30, Loss: 0.6508108675479889, Val Loss: 0.7663657814264297, Learning Rate: 0.1
Epoch 7/30, Loss: 0.6505915224552155, Val Loss: 0.7713499069213867, Learning Rate: 0.1
Epoch 8/30, Loss: 0.6505707085132599, Val Loss: 0.7749321162700653, Learning Rate: 0.010000000000000002
Epoch 9/30, Loss: 0.6329628229141235, Val Loss: 0.7749206572771072, Learning Rate: 0.010000000000000002
Epoch 10/30, Loss: 0.6327947974205017, Val Loss: 0.7749110907316208, Learning Rate: 0.010000000000000002
Epoch 11/30, Loss: 0.6326460301876068, Val Loss: 0.7749034911394119, Learning 